# Module 13 Assignment

## _*Make sure to read the instruction document before beginning the assignment_

In [1]:
# Import libraries

import requests
import json
import pandas as pd
from delta import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, count, avg, sum
from datetime import datetime
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

builder = SparkSession.builder.appName("FoursquareDataEnhanced") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.python.worker.timeout", "120") \
    .config("spark.driver.host", "localhost")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

# Initialize Spark session
# spark = SparkSession.builder \
#     .appName("FoursquareDataEnhanced") \
#     .config("spark.driver.extraJavaOptions", "-Djava.security.manager=allow") \
#     .config("spark.executor.extraJavaOptions", "-Djava.security.manager=allow") \
#     .getOrCreate()
# .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
# spark = SparkSession.builder \
#     .config("spark.driver.host", "localhost") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.python.worker.timeout", "120") \
#     .appName("FoursquareDataEnhanced") \
#     .getOrCreate()

### Question #1 - Initialize Foursquare API (0 points)

In [2]:
# Foursquare API details
api_key = "fsq3tk1Rml2S++Vh6zBYWbYtuigjgDSGxT84gOSBfU4dWyQ="
api_url = "https://api.foursquare.com/v3/places/search"
headers = {
    "Accept": "application/json",
    "Authorization": api_key  #TODO Replace with your Foursquare API Key
}


### Question #2 - Retrieve 1000 Rows of Data (Restaurants) from Foursquare Places in Baltimore (20 points)

In [3]:
# Define search parameters
# Search for restaurants in Baltimore, Maryland
# This will require investigating the data to discover how the fields appear.

# The API is limited to 20 results per request
# If you have limit=20, that means you're getting results 1-20
# Give it offset=21 and you'll get 21-40.

# Parameters for the API request
params = {
    'query': "restaurant", #TODO
    'near': 'Baltimore, MD', #TODO
    'offset': 0,  # Offset the list of returned businesses by this amount.
    'limit': 20   # Set the number of results to retrieve per request (50 is the max)
}

# Make initial API request
response = requests.get(api_url, headers=headers, params=params)
print(response.text)

{"results":[{"fsq_id":"4ad4c014f964a52046ef20e3","categories":[{"id":13236,"name":"Italian Restaurant","short_name":"Italian","plural_name":"Italian Restaurants","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/italian_","suffix":".png"}}],"chains":[],"closed_bucket":"LikelyOpen","distance":515,"geocodes":{"drop_off":{"latitude":39.294326,"longitude":-76.615485},"main":{"latitude":39.294336,"longitude":-76.61532},"roof":{"latitude":39.294336,"longitude":-76.61532}},"link":"/v3/places/4ad4c014f964a52046ef20e3","location":{"address":"405 N Charles St","address_extended":"Ste 1","census_block":"245100401003020","country":"US","dma":"Baltimore","formatted_address":"405 N Charles St, Baltimore, MD 21201","locality":"Baltimore","postcode":"21201","region":"MD"},"name":"Sotto Sopra","related_places":{},"timezone":"America/New_York"},{"fsq_id":"4aff5b30f964a520ac3722e3","categories":[{"id":13079,"name":"Brazilian Restaurant","short_name":"Brazilian","plural_name":"Brazilian Restau

In [4]:
# Create a loop to retrieve the restaurant data by iteratively updating the 'offset' parameter
# Set the maximum number of results to retrieve to 1000
places_data = []
for offset in range(0, 1000, 20):  # Fetch up to 100 results in batches of 20
    params['offset'] = offset
    response = requests.get(api_url, headers=headers, params=params)
    data = response.json()
    for place in data["results"]:  # Adjust "results" based on response
        places_data.append({
            "fsq_id": place.get("fsq_id"),
            "name": place.get("name"),
            "latitude": place["geocodes"]["main"]["latitude"],
            "longitude": place["geocodes"]["main"]["longitude"],
            "distance": place.get("distance"),
            "category_id": place["categories"][0]["id"] if place.get("categories") else None,
            "category_name": place["categories"][0]["name"] if place.get("categories") else None,
            "region": place["location"].get("region"),
            "locality": place["location"].get("locality"),
            "postcode": place["location"].get("postcode"),
            "address": place["location"].get("formatted_address"),
            "timezone": place.get("timezone"),
        })
places_data

[{'fsq_id': '4ad4c014f964a52046ef20e3',
  'name': 'Sotto Sopra',
  'latitude': 39.294336,
  'longitude': -76.61532,
  'distance': 515,
  'category_id': 13236,
  'category_name': 'Italian Restaurant',
  'region': 'MD',
  'locality': 'Baltimore',
  'postcode': '21201',
  'address': '405 N Charles St, Baltimore, MD 21201',
  'timezone': 'America/New_York'},
 {'fsq_id': '4aff5b30f964a520ac3722e3',
  'name': 'Fogo De Chao',
  'latitude': 39.287094,
  'longitude': -76.607334,
  'distance': 556,
  'category_id': 13079,
  'category_name': 'Brazilian Restaurant',
  'region': 'MD',
  'locality': 'Baltimore',
  'postcode': '21202',
  'address': '600 E Pratt St (at Market St), Baltimore, MD 21202',
  'timezone': 'America/New_York'},
 {'fsq_id': '4afdbf3af964a5207e2a22e3',
  'name': 'Trinacria Macaroni Works',
  'latitude': 39.294222,
  'longitude': -76.622827,
  'distance': 998,
  'category_id': 13039,
  'category_name': 'Deli',
  'region': 'MD',
  'locality': 'Baltimore',
  'postcode': '21201',
 

In [5]:
# Transform data into a list of dictionaries with relevant fields

# Fields to Retrieve from Foursquare Places API (1,000 restaurants total):
#   fsq_id
#   name
#   latitude
#   longitude
#   distance
#   category_id
#   category_name
#   region
#   locality
#   postcode
#   address
#   timezone

places = places_data

### Question #3 - Convert to PySpark DataFrame (10 points)

In [6]:
# Convert Python dictionary to PySpark DataFrame
df_restuarants = pd.DataFrame(places_data)
df = spark.createDataFrame(df_restuarants)

In [7]:
# Add a column with today's date (YYYYMMDD) as 'load_dt'
df = df.withColumn("load_dt", lit(datetime.now().strftime("%Y%m%d")))

In [8]:
# Display() the first 5 records of the PySpark DataFrame
df.limit(5).show()

+--------------------+--------------------+---------+----------+--------+-----------+--------------------+------+---------+--------+--------------------+----------------+--------+
|              fsq_id|                name| latitude| longitude|distance|category_id|       category_name|region| locality|postcode|             address|        timezone| load_dt|
+--------------------+--------------------+---------+----------+--------+-----------+--------------------+------+---------+--------+--------------------+----------------+--------+
|4ad4c014f964a5204...|         Sotto Sopra|39.294336| -76.61532|     515|      13236|  Italian Restaurant|    MD|Baltimore|   21201|405 N Charles St,...|America/New_York|20241201|
|4aff5b30f964a520a...|        Fogo De Chao|39.287094|-76.607334|     556|      13079|Brazilian Restaurant|    MD|Baltimore|   21202|600 E Pratt St (a...|America/New_York|20241201|
|4afdbf3af964a5207...|Trinacria Macaron...|39.294222|-76.622827|     998|      13039|               

### Question #4 - Create Bronze Table (20 points)

In [9]:
# Create the database if it doesn't exist
spark.sql("CREATE DATABASE IF NOT EXISTS places_db")

# Switch to the places_db database
spark.sql("USE places_db")

DataFrame[]

In [10]:
# Write a Bronze table with the raw data to the places_db database with the 'delta' format (overwrite if it already exists)
# The table should be named <your_last_name>_bronze, i.e. 'mosko_bronze'
df.write.format("delta").mode("overwrite").saveAsTable(f"places_db.hatzenbeller_bronze")

Py4JJavaError: An error occurred while calling o76.saveAsTable.
: com.google.common.util.concurrent.ExecutionError: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at com.google.common.cache.LocalCache$Segment.get(LocalCache.java:2261)
	at com.google.common.cache.LocalCache.get(LocalCache.java:4000)
	at com.google.common.cache.LocalCache$LocalManualCache.get(LocalCache.java:4789)
	at org.apache.spark.sql.delta.DeltaLog$.getDeltaLogFromCache$1(DeltaLog.scala:865)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:875)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:784)
	at org.apache.spark.sql.delta.DeltaLog$.forTable(DeltaLog.scala:746)
	at org.apache.spark.sql.delta.catalog.DeltaCatalog.$anonfun$createDeltaTable$10(DeltaCatalog.scala:186)
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.sql.delta.catalog.DeltaCatalog.$anonfun$createDeltaTable$1(DeltaCatalog.scala:184

In [ ]:
# Query all records of the bronze table you created above
bronze_table = spark.sql("SELECT * FROM places_db.hatzenbeller_bronze")

# display() the first 5 records
bronze_table.limit(5).show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `places_db`.`hatzenbeller_bronze` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [places_db, hatzenbeller_bronze], [], false


### Question #5 - Create Silver Table (20 points)

In [ ]:
# Step 1: Select the relevant columns from the Bronze table
silver_df = spark.sql("""
    SELECT fsq_id, name, distance, category_name, region, locality, postcode, address 
    FROM places_db.hatzenbeller_bronze
""")

# Step 2: Remove duplicate rows from the Silver DataFrame
silver_df = silver_df.dropDuplicates()

# Show the cleaned Silver DataFrame
silver_df.show()


Py4JJavaError: An error occurred while calling o37.sql.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at io.delta.storage.HadoopFileSystemLogStore.listFrom(HadoopFileSystemLogStore.java:59)
	at org.apache.spark.sql.delta.storage.LogStoreAdaptor.listFrom(LogStore.scala:452)
	at org.apache.spark.sql.delta.storage.DelegatingLogStore.li

In [ ]:
# Write a Silver table to the places_db database with the 'delta' format (overwrite if it already exists)
# The table should be named <your_last_name>_silver, i.e. 'mosko_silver'
#TODO

In [ ]:
# Query all records of the silver table you created above
silver_table = #TODO

# display() the first 5 records
#TODO

### Question #6 - Create Gold Table (20 points)

In [ ]:
# Write a PySpark query to group the Silver DataFrame (silver_df) by three columns:
#   category_name: The name of the category (e.g., "Restaurant").
#   region: The state or region of the place.
#   locality: The city or locality of the place.
#
# Calculate the following aggregations for each group:
#   num_places: The total number of places in each group (count of fsq_id).
#   avg_distance: The average distance of the places from the query location.
#   unique_postcodes: The number of unique postcodes in each group.
#
# Store the aggregated results in a new DataFrame named gold_df

gold_df = #TODO

In [ ]:
# Write a Gold table to the places_db database with the 'delta' format (overwrite if it already exists)
# The table should be named <your_last_name>_gold, i.e. 'mosko_gold'
#TODO

## Question #7 - Query the Gold Table (10 points)

In [ ]:
# Query the gold table and store results in a PySpark DataFrame with an avg_distance of less than 3000
query = #TODO
gold_table = spark.sql(query)

# display() all records
#TODO

## Submitting the Assignment
Submit your Jupyter Notebook file with **all cells executed and outputs displayed**.

Export the notebook from Databricks as an html file (File > Export > HTML) and upload to Canvas: \<student-first-intial>\<student-last-name>\-module13.html
    
Upload a Jupyter Notebook with your code: \<student-first-intial>\<student-last-name>\-module13.ipyn